In [1]:
import numpy as np
import pandas as pd
from __future__ import print_function

from sklearn.ensemble  import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [4]:
# the directory that contains all the files
dataDirectory = 'data/'

In [5]:
file = 'llcp2017_formatted.csv'
df = pd.read_csv(dataDirectory+file)

/home/urwa/miniconda3/envs/gpd/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,170,174) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.head()

,STATE FIPS CODE,FILE MONTH,INTERVIEW DATE,INTERVIEW MONTH,INTERVIEW DAY,INTERVIEW YEAR,FINAL DISPOSITION,ANNUAL SEQUENCE NUMBER,PRIMARY SAMPLING UNIT,CORRECT TELEPHONE NUMBER?,...,_IMPCAGE,_IMPCRAC,_IMPCSEX,_IMPEDUC,_IMPHOME,_IMPMRTL,_IMPNPH,_IMPSEX,_M_RACE,_URBNRRL
0,Alabama,January,1302017,01,30,2017,1100,2017000001,2017000001,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
1,Alabama,January,1122017,01,12,2017,1100,2017000002,2017000002,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
2,Alabama,January,1102017,01,10,2017,1100,2017000003,2017000003,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
3,Alabama,January,2082017,02,8,2017,1200,2017000004,2017000004,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
4,Alabama,January,1302017,01,30,2017,1100,2017000005,2017000005,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing


In [7]:
print("Number of records: ",len(df))

Number of records:  450642


In [8]:
df['INTERVIEW YEAR'].value_counts()/np.sum(df['INTERVIEW YEAR'].value_counts())

2017    0.973964
2018    0.026036
Name: INTERVIEW YEAR, dtype: float64

#### Most of the data is from 2017

In [9]:
list(np.sort(df.columns))

['(EVER TOLD) YOU HAD ANY OTHER TYPES OF C',
 '(EVER TOLD) YOU HAD SKIN CANCER?',
 '(EVER TOLD) YOU HAVE (COPD) CHRONIC OBST',
 '(EVER TOLD) YOU HAVE DIABETES',
 '(EVER TOLD) YOU HAVE KIDNEY DISEASE?',
 '150 MINUTE PHYSICAL ACTIVITY CALCULATED',
 '300 MINUTE PHYSICAL ACTIVITY 2-LEVEL CAL',
 '300 MINUTE PHYSICAL ACTIVITY CALCULATED',
 'ACTIVITIES LIMITED DUE TO ASTHMA DURING',
 'ACTIVITY MET VALUE FOR FIRST ACTIVITY',
 'ACTIVITY MET VALUE FOR SECOND ACTIVITY',
 'ADLTCHLD',
 'ADULT FLU SHOT/SPRAY PAST 12 MOS',
 'ADULTS WITH GOOD OR BETTER HEALTH',
 'AEROBIC AND STRENGTHENING (2-LEVEL)',
 'AEROBIC AND STRENGTHENING GUIDELINE',
 'AGE',
 'AGE AT ASTHMA DIAGNOSIS',
 'AGE TOLD HAD CANCER',
 'AGE WHEN TOLD DIABETIC',
 'ALWAYS OR NEARLY ALWAYS WEAR SEAT BELTS',
 'ALWAYS WEAR SEAT BELTS',
 'ANNUAL SEQUENCE NUMBER',
 'ANY FIREARMS IN HOME',
 'ANY FIREARMS LOADED',
 'ANY LOADED FIREARMS ALSO UNLOCKED',
 'ARE YOU 18 YEARS OF AGE OR OLDER?',
 'ARE YOU 18 YEARS OF AGE OR OLDER?.1',
 'ARE YOU A VETERA

In [14]:
# Find the columns that contain smoking key word

filterList = ['smok','cig']
def containsKeyword(sent):
    sent = sent.lower()
    for f in filterList:
        if f in sent:
            return True
    return False
    
smokecols = [c for c in df.columns if containsKeyword(c)]
smokecols

['SMOKED AT LEAST 100 CIGARETTES',
 'FREQUENCY OF DAYS NOW SMOKING',
 'STOPPED SMOKING IN PAST 12 MONTHS',
 'INTERVAL SINCE LAST SMOKED',
 'USE OF SMOKELESS TOBACCO PRODUCTS',
 'HAVE YOU EVER USED AN E-CIGARETTE OR OTH',
 'DO YOU NOW USE E-CIGARETTES, EVERY DAY,',
 'HOW MANY YEARS HAVE YOU SMOKED TOBACCO P',
 'HOW OLD WHEN YOU FIRST STARTED SMOKING?',
 'HOW OLD WHEN YOU LAST SMOKED?',
 'ON AVERAGE, HOW MANY CIGARETTES DO YOU S',
 'COMPUTED SMOKING STATUS',
 'CURRENT SMOKING CALCULATED VARIABLE',
 'COMPUTED E-CIGARETTE USER STATUS',
 'CURRENT E-CIGARETTE USER CALCULATED VARI']

In [15]:
# See some values in smoking columns
df[smokecols][:5]

,SMOKED AT LEAST 100 CIGARETTES,FREQUENCY OF DAYS NOW SMOKING,STOPPED SMOKING IN PAST 12 MONTHS,INTERVAL SINCE LAST SMOKED,USE OF SMOKELESS TOBACCO PRODUCTS,HAVE YOU EVER USED AN E-CIGARETTE OR OTH,"DO YOU NOW USE E-CIGARETTES, EVERY DAY,",HOW MANY YEARS HAVE YOU SMOKED TOBACCO P,HOW OLD WHEN YOU FIRST STARTED SMOKING?,HOW OLD WHEN YOU LAST SMOKED?,"ON AVERAGE, HOW MANY CIGARETTES DO YOU S",COMPUTED SMOKING STATUS,CURRENT SMOKING CALCULATED VARIABLE,COMPUTED E-CIGARETTE USER STATUS,CURRENT E-CIGARETTE USER CALCULATED VARI
0,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not at all,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Never smoked,No,Never used E-cigarettes,Not currently using E-cigarettes
1,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not at all,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Never smoked,No,Never used E-cigarettes,Not currently using E-cigarettes
2,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not at all,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Never smoked,No,Never used E-cigarettes,Not currently using E-cigarettes
3,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not at all,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Never smoked,No,Never used E-cigarettes,Not currently using E-cigarettes
4,Yes,Not at all,Not asked or Missing,Within the past month (less than 1 month ago),Not at all,No,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Not asked or Missing,Former smoker,No,Never used E-cigarettes,Not currently using E-cigarettes


In [16]:
# See the frequency distribution of smoking columns
for c in smokecols:
    print(c)
    print(df[c].value_counts())
    print()

SMOKED AT LEAST 100 CIGARETTES
No                      246459
Yes                     185837
Not asked or Missing     15962
Don�t know/Not Sure       1938
Refused                    446
Name: SMOKED AT LEAST 100 CIGARETTES, dtype: int64

FREQUENCY OF DAYS NOW SMOKING
Not asked or Missing    264878
Not at all              121867
Every day                44680
Some days                18882
Refused                    237
Don�t Know/Not Sure         98
Name: FREQUENCY OF DAYS NOW SMOKING, dtype: int64

STOPPED SMOKING IN PAST 12 MONTHS
Not asked or Missing    387131
Yes                      35498
No                       27796
Don�t know/Not Sure        178
Refused                     39
Name: STOPPED SMOKING IN PAST 12 MONTHS, dtype: int64

INTERVAL SINCE LAST SMOKED
Not asked or Missing                                              328840
10 years or more                                                   80302
Within the past 5 years (1 year but less than 5 years ago)         16240
Withi

### The computed and calculated columns are cleaned data and have fewer missing values

In [17]:
## Explore the calculated and computed columns
filterList = ['computed', 'calculated']
def containsKeyword(sent):
    sent = sent.lower()
    for f in filterList:
        if f in sent:
            return True
    return False
    
cols = [c for c in df.columns if containsKeyword(c)]
print("Number of calculated/computed columns: ",len(cols))
cols

Number of calculated/computed columns:  41


['COMPUTED PHYSICAL HEALTH STATUS',
 'COMPUTED MENTAL HEALTH STATUS',
 'HIGH BLOOD PRESSURE CALCULATED VARIABLE',
 'CHOLESTEROL CHECKED CALCULATED VARIABLE',
 'HIGH CHOLESTEROL CALCULATED VARIABLE',
 'LIFETIME ASTHMA CALCULATED VARIABLE',
 'CURRENT ASTHMA CALCULATED VARIABLE',
 'COMPUTED ASTHMA STATUS',
 'COMPUTED PREFERRED RACE',
 'CALCULATED NON-HISPANIC RACE INCLUDING M',
 'COMPUTED RACE-ETHNICITY GROUPING',
 'COMPUTED NON-HISPANIC WHITES/ALL OTHERS',
 'COMPUTED FIVE LEVEL RACE/ETHNICITY CATEG',
 'COMPUTED RACE GROUPS USED FOR INTERNET P',
 'COMPUTED HEIGHT IN INCHES',
 'COMPUTED HEIGHT IN METERS',
 'COMPUTED WEIGHT IN KILOGRAMS',
 'COMPUTED BODY MASS INDEX',
 'COMPUTED BODY MASS INDEX CATEGORIES',
 'OVERWEIGHT OR OBESE CALCULATED VARIABLE',
 'COMPUTED NUMBER OF CHILDREN IN HOUSEHOLD',
 'COMPUTED LEVEL OF EDUCATION COMPLETED CA',
 'COMPUTED INCOME CATEGORIES',
 'COMPUTED SMOKING STATUS',
 'CURRENT SMOKING CALCULATED VARIABLE',
 'COMPUTED E-CIGARETTE USER STATUS',
 'CURRENT E-CIGARET

### These 41 columns calculated columns cover most socieconomic features so they should be used for initial models

In [18]:
for c in cols:
    print(c)
    print(df[c].value_counts())
    print()

COMPUTED PHYSICAL HEALTH STATUS
Zero days when physical health not good    276509
1-13 days when physical health not good    103480
14+ days when physical health not good      61308
Don�t know/Refused/Missing                   9345
Name: COMPUTED PHYSICAL HEALTH STATUS, dtype: int64

COMPUTED MENTAL HEALTH STATUS
Zero days when mental health not good    300525
1-13 days when mental health not good     93057
14+ days when mental health not good      49846
Don�t know/Refused/Missing                 7214
Name: COMPUTED MENTAL HEALTH STATUS, dtype: int64

HIGH BLOOD PRESSURE CALCULATED VARIABLE
No                                     267905
Yes                                    181337
Don�t know/Not Sure/Refused/Missing      1400
Name: HIGH BLOOD PRESSURE CALCULATED VARIABLE, dtype: int64

CHOLESTEROL CHECKED CALCULATED VARIABLE
Had cholesterol checked in past 5 years             385747
Have never had cholesterol checked                   26532
Don�t know/Not Sure Or Refused/Missing       

No                            368326
Yes                            63562
Don�t know/Refused/Missing     18754
Name: CURRENT SMOKING CALCULATED VARIABLE, dtype: int64

COMPUTED E-CIGARETTE USER STATUS
Never used E-cigarettes                      363247
Former E-cigarette user                       53028
Don�t know/Refused/Missing                    20678
Current E-cigarette user - uses some days      8667
Current E-cigarette user - uses every day      5022
Name: COMPUTED E-CIGARETTE USER STATUS, dtype: int64

CURRENT E-CIGARETTE USER CALCULATED VARI
Not currently using E-cigarettes    416275
Don�t know/Refused/Missing           20678
Current E-cigarette user             13689
Name: CURRENT E-CIGARETTE USER CALCULATED VARI, dtype: int64

COMPUTED DRINK-OCCASIONS-PER-DAY
Drink-Occasions per day                   221510
No Drink-Occasions per day                203392
Don�t know/Not Sure Or Refused/Missing     25740
Name: COMPUTED DRINK-OCCASIONS-PER-DAY, dtype: int64

BINGE DRINKING CALC

Name: FLU SHOT CALCULATED VARIABLE, dtype: int64

PNEUMONIA VACCINATION CALCULATED VARIABL
Age Less Than 65                          287849
Yes                                       107390
No                                         33679
Don�t know/Not Sure Or Refused/Missing     21724
Name: PNEUMONIA VACCINATION CALCULATED VARIABL, dtype: int64

EVER BEEN TESTED FOR HIV CALCULATED VARI
No                             268409
Yes                            127137
Not asked or missing            41727
Don�t know/Not Sure/Refused     13369
Name: EVER BEEN TESTED FOR HIV CALCULATED VARI, dtype: int64



In [20]:
# create a datframe that only has calculated columns
comDF = df[cols]
comDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450642 entries, 0 to 450641
Data columns (total 41 columns):
COMPUTED PHYSICAL HEALTH STATUS             450642 non-null object
COMPUTED MENTAL HEALTH STATUS               450642 non-null object
HIGH BLOOD PRESSURE CALCULATED VARIABLE     450642 non-null object
CHOLESTEROL CHECKED CALCULATED VARIABLE     450642 non-null object
HIGH CHOLESTEROL CALCULATED VARIABLE        450642 non-null object
LIFETIME ASTHMA CALCULATED VARIABLE         450642 non-null object
CURRENT ASTHMA CALCULATED VARIABLE          450642 non-null object
COMPUTED ASTHMA STATUS                      450642 non-null object
COMPUTED PREFERRED RACE                     450642 non-null object
CALCULATED NON-HISPANIC RACE INCLUDING M    450642 non-null object
COMPUTED RACE-ETHNICITY GROUPING            450642 non-null object
COMPUTED NON-HISPANIC WHITES/ALL OTHERS     450642 non-null object
COMPUTED FIVE LEVEL RACE/ETHNICITY CATEG    450642 non-null object
COMPUTED RACE GROU

In [124]:
print(pd.Series.idxmin(comDF['COMPUTED PHYSICAL HEALTH STATUS'].value_counts()))
comDF['COMPUTED PHYSICAL HEALTH STATUS'].value_counts()

Don�t know/Refused/Missing


Zero days when physical health not good    232663
1-13 days when physical health not good     89516
14+ days when physical health not good      51440
Don�t know/Refused/Missing                   5973
Name: COMPUTED PHYSICAL HEALTH STATUS, dtype: int64

In [101]:
comDF.describe()

,COMPUTED WEIGHT IN KILOGRAMS,COMPUTED FRUIT JUICE INTAKE IN TIMES PER,COMPUTED FRUIT INTAKE IN TIMES PER DAY,COMPUTED DARK GREEN VEGETABLE INTAKE IN,COMPUTED POTATO SERVINGS PER DAY,COMPUTED OTHER VEGETABLE INTAKE IN TIMES
count,420723.000000,417359.000000,416996.000000,417323.000000,413900.000000,413348.000000
mean,8158.443192,33.795399,116.103440,61.060855,25.512600,100.917307
std,2069.914152,103.738742,170.791619,133.629245,64.430663,201.645847
min,2268.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6759.000000,0.000000,43.000000,14.000000,7.000000,43.000000
50%,7938.000000,7.000000,100.000000,43.000000,14.000000,100.000000
75%,9163.000000,43.000000,200.000000,100.000000,29.000000,100.000000
max,27500.000000,9900.000000,9900.000000,9900.000000,9900.000000,9900.000000


In [96]:
print("Data types and unique values")
[ (comDF[c].dtype.name,len(comDF[c].value_counts()))  for c in comDF.columns]

Data types and unique values


[('object', 4),
 ('object', 4),
 ('object', 3),
 ('object', 4),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('object', 4),
 ('object', 10),
 ('object', 10),
 ('object', 9),
 ('object', 3),
 ('object', 6),
 ('object', 6),
 ('object', 2),
 ('object', 2),
 ('float64', 561),
 ('object', 2),
 ('object', 5),
 ('object', 3),
 ('object', 7),
 ('object', 5),
 ('object', 6),
 ('object', 5),
 ('object', 3),
 ('object', 5),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('float64', 103),
 ('float64', 128),
 ('float64', 119),
 ('float64', 92),
 ('float64', 120),
 ('object', 4),
 ('object', 4),
 ('object', 4),
 ('object', 4),
 ('object', 4)]

In [103]:
print("NANs for numeric variables")
[ (c, np.sum(np.isnan(comDF[c])))  for c in comDF.columns if np.issubdtype(comDF[c].dtype, np.number) ]

NANs for numeric variables


[('COMPUTED WEIGHT IN KILOGRAMS', 29919),
 ('COMPUTED FRUIT JUICE INTAKE IN TIMES PER', 33283),
 ('COMPUTED FRUIT INTAKE IN TIMES PER DAY', 33646),
 ('COMPUTED DARK GREEN VEGETABLE INTAKE IN', 33319),
 ('COMPUTED POTATO SERVINGS PER DAY', 36742),
 ('COMPUTED OTHER VEGETABLE INTAKE IN TIMES', 37294)]

In [110]:
print("Records: ",len(comDF))
comDF = comDF.dropna()
print("Records after cleaning Nan: ",len(comDF))

Records:  379592
Records after cleaning Nan:  379592


In [109]:
print("Infintes for numeric variables")
[ (c, np.sum(np.isinf(comDF[c])))  for c in comDF.columns if np.issubdtype(comDF[c].dtype, np.number) ]

Infintes for numeric variables


[('COMPUTED WEIGHT IN KILOGRAMS', 0),
 ('COMPUTED FRUIT JUICE INTAKE IN TIMES PER', 0),
 ('COMPUTED FRUIT INTAKE IN TIMES PER DAY', 0),
 ('COMPUTED DARK GREEN VEGETABLE INTAKE IN', 0),
 ('COMPUTED POTATO SERVINGS PER DAY', 0),
 ('COMPUTED OTHER VEGETABLE INTAKE IN TIMES', 0)]

In [93]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
def labelEncodeCategoricalFeatures(DF):
    df = DF.copy()
    for c in df.columns:
        if df[c].dtype.name == 'object':
            df[c] = labelencoder.fit_transform(df[c])
    return df

In [164]:
# remove class imballance by sampling from majority class
def classImballanceDownSample(df,ycol):
    df = df.copy()
    valueCount = df[ycol].value_counts()
    print("Before Class Imballance Treatment: ")
    print(valueCount)
    classes = valueCount.index
    counts = valueCount.values
    minClassSize = np.min(counts)
    for clas in classes:
        df1 = df[df[ycol]==clas]
        df2 = df[df[ycol]!=clas]
        
        df1 = df1.sample(n=minClassSize, random_state=50)
        df = df1.append(df2)
    #shuffling the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    print("After Class Imballance Treatment: ")
    print(df[ycol].value_counts())
    return df  

In [165]:
# remove class imballance by up sampling minority classes
def classImballanceUpSample(df,ycol):
    df = df.copy()
    valueCount = df[ycol].value_counts()
    print("Before Class Imballance Treatment: ")
    print(valueCount)
    classes = valueCount.index
    counts = valueCount.values
    maxClassSize = np.max(counts).astype(float)
    fractions = maxClassSize / counts
    for clas,f in zip(classes,fractions):
        df1 = df[df[ycol]==clas]
        df2 = df[df[ycol]!=clas]
        
        df1 = df1.sample(frac=f, replace=True, random_state=50)
        df = df1.append(df2)
    #shuffling the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    print("After Class Imballance Treatment: ")
    print(df[ycol].value_counts())
    return df  

In [240]:
# model that trains a default random forest and prints the report
def getModel(df, ballanceClasses=None, rfcBalance = None):
    
    xcols = df.columns[:len(df.columns)-1]
    ycols = [df.columns[-1]]
    print("Predictors: ",len(xcols))
    print("label: ",ycols[0])
    
#     if ballanceClasses != None:
#         df = ballanceClasses(df, ycols[0])
        
    X = df[xcols]
    Y = df[ycols]
    
    X = labelEncodeCategoricalFeatures(X)
    print("Label Encoding.")
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, shuffle=True)
    print("Train Test Split:")
    print(np.array(X_train).shape, np.array(X_test).shape, np.array(y_train).shape, np.array(y_test).shape)
    
    if ballanceClasses != None:
        dfTrain = X_train
        dfTrain['label'] = y_train
        dfTrain = ballanceClasses(dfTrain,'label')
        X_train = dfTrain[ [c for c in dfTrain.columns if c not in 'label'] ]
        y_train = dfTrain['label']
    
    #print("Checking if sampling worked:")
    #print(y_train.value_counts())
    #print()
    
    if rfcBalance == None:
        clf = rfc()
    else:
        clf = rfc(class_weight = rfcBalance)
        print(clf)
    
    #clf = rfc()
    clf.fit(X_train, y_train.values.ravel())
    y_pred = clf.predict(X_test)
    print("Accuracy : ", np.sum(y_test[ycols[0]].values==y_pred).astype(float)/len(y_test) )
    print(classification_report(y_test[ycols[0]].values, y_pred))

### Predicting current calculated variables

In [208]:
## Predicting current calculated variables
xcols = [c for c in cols if c not in smokecols] + ['CURRENT SMOKING CALCULATED VARIABLE']
doesSmokeDf = comDF[xcols]
print("Original: ",len(doesSmokeDf))
doesSmokeDf = doesSmokeDf[doesSmokeDf['CURRENT SMOKING CALCULATED VARIABLE'].apply(lambda x: x in ['Yes','No'])]
print("Cleaned Label: ",len(doesSmokeDf))

print("Columns :",len(doesSmokeDf.columns))
print()
print(doesSmokeDf['CURRENT SMOKING CALCULATED VARIABLE'].value_counts())

Original:  379592
Cleaned Label:  377922
Columns : 38

No     322389
Yes     55533
Name: CURRENT SMOKING CALCULATED VARIABLE, dtype: int64


In [209]:
## Without removing the class imballance
getModel(doesSmokeDf)

Predictors:  37
label:  CURRENT SMOKING CALCULATED VARIABLE
Label Encoding.
Train Test Split:
(253207, 37) (124715, 37) (253207, 1) (124715, 1)
Accuracy :  0.851814136231
              precision    recall  f1-score   support

          No       0.87      0.98      0.92    106363
         Yes       0.49      0.13      0.20     18352

   micro avg       0.85      0.85      0.85    124715
   macro avg       0.68      0.55      0.56    124715
weighted avg       0.81      0.85      0.81    124715



### Class imballance is influencing the model to converge towards a biased model. We need to treat class imballance.

### Upsampling before test train split gives extremely high accuracy but it is not correct. Because a lot of training examples show up in test and thus we are not actually measuring out of sample performance. So I changed code to treat imballance only in the train data.

In [211]:
## With upsampling the minority class
getModel(doesSmokeDf, classImballanceUpSample)

Predictors:  37
label:  CURRENT SMOKING CALCULATED VARIABLE
Label Encoding.
Train Test Split:
(253207, 37) (124715, 37) (253207, 1) (124715, 1)
Before Class Imballance Treatment: 
No     216026
Yes     37181
Name: label, dtype: int64


/home/urwa/miniconda3/envs/gpd/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


After Class Imballance Treatment: 
Yes    216026
No     216026
Name: label, dtype: int64
Accuracy :  0.835152146895
              precision    recall  f1-score   support

          No       0.88      0.93      0.91    106363
         Yes       0.41      0.27      0.33     18352

   micro avg       0.84      0.84      0.84    124715
   macro avg       0.65      0.60      0.62    124715
weighted avg       0.81      0.84      0.82    124715



In [212]:
## With downsampling the majority class
getModel(doesSmokeDf, classImballanceDownSample)

Predictors:  37
label:  CURRENT SMOKING CALCULATED VARIABLE
Label Encoding.
Train Test Split:
(253207, 37) (124715, 37) (253207, 1) (124715, 1)
Before Class Imballance Treatment: 
No     216026
Yes     37181
Name: label, dtype: int64


/home/urwa/miniconda3/envs/gpd/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


After Class Imballance Treatment: 
Yes    37181
No     37181
Name: label, dtype: int64
Accuracy :  0.711173475524
              precision    recall  f1-score   support

          No       0.92      0.73      0.81    106363
         Yes       0.28      0.62      0.39     18352

   micro avg       0.71      0.71      0.71    124715
   macro avg       0.60      0.67      0.60    124715
weighted avg       0.82      0.71      0.75    124715



### COMPUTED SMOKING STATUS

In [213]:
## Predicting current calculated variables
xcols = [c for c in cols if c not in smokecols] + ['COMPUTED SMOKING STATUS']
quitSmokeDf = comDF[xcols]
print("Original: ",len(quitSmokeDf))
quitSmokeDf = quitSmokeDf[quitSmokeDf['COMPUTED SMOKING STATUS'].apply(lambda x: x in  \
                                                                       ['Former smoker', \
                                                                        'Current smoker - now smokes every day',\
                                                                      'Current smoker - now smokes some days'])]
quitSmokeDf['COMPUTED SMOKING STATUS'] = quitSmokeDf['COMPUTED SMOKING STATUS'].apply(lambda x: x if 'Current smoker' not in x \
                                                                                     else 'Current smoker')

print("Cleaned Label: ",len(quitSmokeDf))

print("Columns :",len(quitSmokeDf.columns))
print()
print(quitSmokeDf['COMPUTED SMOKING STATUS'].value_counts())

Original:  379592
Cleaned Label:  162967
Columns : 38

Former smoker     107434
Current smoker     55533
Name: COMPUTED SMOKING STATUS, dtype: int64


In [214]:
## Without removing the class imballance
getModel(quitSmokeDf)

Predictors:  37
label:  COMPUTED SMOKING STATUS
Label Encoding.
Train Test Split:
(109187, 37) (53780, 37) (109187, 1) (53780, 1)
Accuracy :  0.696653030866
                precision    recall  f1-score   support

Current smoker       0.55      0.54      0.55     18151
 Former smoker       0.77      0.77      0.77     35629

     micro avg       0.70      0.70      0.70     53780
     macro avg       0.66      0.66      0.66     53780
  weighted avg       0.70      0.70      0.70     53780



In [215]:
## With upsampling the minority class
getModel(quitSmokeDf, classImballanceUpSample)

Predictors:  37
label:  COMPUTED SMOKING STATUS
Label Encoding.
Train Test Split:
(109187, 37) (53780, 37) (109187, 1) (53780, 1)
Before Class Imballance Treatment: 
Former smoker     71805
Current smoker    37382
Name: label, dtype: int64


/home/urwa/miniconda3/envs/gpd/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


After Class Imballance Treatment: 
Current smoker    71805
Former smoker     71805
Name: label, dtype: int64
Accuracy :  0.671755299368
                precision    recall  f1-score   support

Current smoker       0.51      0.66      0.58     18151
 Former smoker       0.80      0.68      0.73     35629

     micro avg       0.67      0.67      0.67     53780
     macro avg       0.65      0.67      0.65     53780
  weighted avg       0.70      0.67      0.68     53780



In [216]:
## With downsampling the majority class
getModel(quitSmokeDf, classImballanceDownSample)

Predictors:  37
label:  COMPUTED SMOKING STATUS
Label Encoding.
Train Test Split:
(109187, 37) (53780, 37) (109187, 1) (53780, 1)
Before Class Imballance Treatment: 
Former smoker     71805
Current smoker    37382
Name: label, dtype: int64


/home/urwa/miniconda3/envs/gpd/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


After Class Imballance Treatment: 
Current smoker    37382
Former smoker     37382
Name: label, dtype: int64
Accuracy :  0.645537374489
                precision    recall  f1-score   support

Current smoker       0.48      0.74      0.58     18151
 Former smoker       0.82      0.60      0.69     35629

     micro avg       0.65      0.65      0.65     53780
     macro avg       0.65      0.67      0.64     53780
  weighted avg       0.70      0.65      0.66     53780



### Explore E ciggerete smoking

In [225]:
ecigdf = comDF.copy()
ecigdf.rename(columns={'CURRENT E-CIGARETTE USER CALCULATED VARI': 'esmoke', \
                     'CURRENT SMOKING CALCULATED VARIABLE':'smoke'},inplace = True)


mapper = {'Current E-cigarette user': 'Yes', 'Not currently using E-cigarettes': 'No' }
ecigdf['esmoke'] = ecigdf['esmoke'].map(mapper)

print("data size: ",len(ecigdf))
ecigdf = ecigdf[ecigdf.smoke.apply(lambda x: x in ['Yes','No']) ]
ecigdf = ecigdf[ecigdf.esmoke.apply(lambda x: x in ['Yes','No']) ]
print("clean data size: ",len(ecigdf))

print("Frequency distributions:")
print(ecigdf.esmoke.value_counts())
print(ecigdf.smoke.value_counts())

data size:  379592
clean data size:  377435
Frequency distributions:
No     365208
Yes     12227
Name: esmoke, dtype: int64
No     321970
Yes     55465
Name: smoke, dtype: int64


In [226]:
print("Just smoke :", np.sum((ecigdf.smoke == 'Yes') & (ecigdf.esmoke == 'No')))
print("Just esmoke:", np.sum((ecigdf.smoke == 'No') & (ecigdf.esmoke == 'Yes')))
print("SMoke Both :", np.sum((ecigdf.smoke == 'Yes') & (ecigdf.esmoke == 'Yes')))
print("Smoke none :", np.sum((ecigdf.smoke == 'No') & (ecigdf.esmoke == 'No')))

Just smoke : 48998
Just esmoke: 5760
SMoke Both : 6467
Smoke none : 316210


### Assumption: Considering esmokers and those who smoke both as esmokers.

In [227]:
# remove non smokers
ecigdf = ecigdf[ (ecigdf.smoke == 'Yes') | (ecigdf.esmoke == 'Yes') ]
print("clean data size: ",len(ecigdf))

clean data size:  61225


In [228]:
# label is 1 for esmoker and zero for non smoker
ecigdf['label'] = ecigdf.esmoke == 'Yes'

In [229]:
excludecolumns = smokecols + ['esmoke', 'smoke']
ecigdf = ecigdf[[c for c in ecigdf.columns if c not in excludecolumns ]]
print('Columns : ',len(ecigdf.columns))
print("Frequency distributions:")
print(ecigdf.label.value_counts())

Columns :  38
Frequency distributions:
False    48998
True     12227
Name: label, dtype: int64


In [231]:
## Without removing the class imballance
getModel(ecigdf)

Predictors:  37
label:  label
Label Encoding.
Train Test Split:
(41020, 37) (20205, 37) (41020, 1) (20205, 1)
Accuracy :  0.787824795843
              precision    recall  f1-score   support

       False       0.80      0.98      0.88     16115
        True       0.31      0.04      0.07      4090

   micro avg       0.79      0.79      0.79     20205
   macro avg       0.56      0.51      0.48     20205
weighted avg       0.70      0.79      0.72     20205



In [232]:
## With upsampling the minority class
getModel(ecigdf, classImballanceUpSample)

Predictors:  37
label:  label
Label Encoding.
Train Test Split:
(41020, 37) (20205, 37) (41020, 1) (20205, 1)
Before Class Imballance Treatment: 
False    32883
True      8137
Name: label, dtype: int64
After Class Imballance Treatment: 
True     32883
False    32883
Name: label, dtype: int64


/home/urwa/miniconda3/envs/gpd/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Accuracy :  0.74852759218
              precision    recall  f1-score   support

       False       0.81      0.90      0.85     16115
        True       0.29      0.16      0.21      4090

   micro avg       0.75      0.75      0.75     20205
   macro avg       0.55      0.53      0.53     20205
weighted avg       0.70      0.75      0.72     20205



In [237]:
## With downsampling the majority class
getModel(ecigdf, classImballanceDownSample)

Predictors:  37
label:  label
Label Encoding.
Train Test Split:
(41020, 37) (20205, 37) (41020, 1) (20205, 1)
Before Class Imballance Treatment: 
False    32883
True      8137
Name: label, dtype: int64
After Class Imballance Treatment: 
True     8137
False    8137
Name: label, dtype: int64


/home/urwa/miniconda3/envs/gpd/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Accuracy :  0.60272209849
              precision    recall  f1-score   support

       False       0.82      0.64      0.72     16115
        True       0.25      0.47      0.32      4090

   micro avg       0.60      0.60      0.60     20205
   macro avg       0.54      0.55      0.52     20205
weighted avg       0.71      0.60      0.64     20205



In [252]:
## Use rfc weight paramter to ballance “balanced”, “balanced_subsample”
weights = {True:4,False:1}
getModel(ecigdf, ballanceClasses = None, rfcBalance = weights)

Predictors:  37
label:  label
Label Encoding.
Train Test Split:
(41020, 37) (20205, 37) (41020, 1) (20205, 1)
RandomForestClassifier(bootstrap=True, class_weight={False: 1, True: 4},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators='warn', n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)
Accuracy :  0.791635733729
              precision    recall  f1-score   support

       False       0.80      0.98      0.88     16115
        True       0.35      0.03      0.06      4090

   micro avg       0.79      0.79      0.79     20205
   macro avg       0.58      0.51      0.47     20205
weighted avg       0.71      0.79      0.72     20205



### Weights argument is not solving the imballance problem.

### Some Rules of Thumb for Imballance

    - Consider testing under-sampling when you have an a lot data (tens- or hundreds of thousands of instances or more)
    - Consider testing over-sampling when you don’t have a lot of data (tens of thousands of records or less)
    - Consider testing random and non-random (e.g. stratified) sampling schemes.
    - Consider testing different resampled ratios (e.g. you don’t have to target a 1:1 ratio in a binary classification problem, try other ratios)


### ALso try SMOTE